In [46]:
import torch as t
from torch.nn.functional import normalize
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import SGDRegressor
from sklearn.multioutput import MultiOutputRegressor
import torch.utils.data as D
from torch.utils.data import TensorDataset

## Dataset preparations

In [47]:
dataset = t.load("data/ffa_channelpark.pt", weights_only=False)
observations,_ = dataset.tensors
positions = observations[:,:3].float()

angles =  10 * positions[:, 2] * t.pi / 180
cosi = t.stack([t.cos(angles), t.sin(angles)], dim=1)
positions = t.cat((positions[:, :2], cosi,positions[:,3:]), dim=1)

#scaler = StandardScaler()
#positions_scaled = t.tensor(scaler.fit_transform(positions))
positions_scaled = normalize(positions,dim=0)

X = positions[:-1].unfold(dimension=0, size=10, step=1).flatten(start_dim=1,end_dim=2)
y =  positions[10:]  

X_scaled = positions_scaled[:-1].unfold(dimension=0, size=10, step=1).flatten(start_dim=1,end_dim=2)
y_scaled =  positions_scaled[10:]  

print("Data shape:\t",positions.shape)
print("Input shape:\t",X.shape)
print("Output shape:\t",y.shape)

Data shape:	 torch.Size([2592000, 4])
Input shape:	 torch.Size([2591990, 40])
Output shape:	 torch.Size([2591990, 4])


## Constant model

In [48]:
mse  = mean_squared_error(positions[1:], positions[:-1])
print("Mean squared error:\t",mse)

Mean squared error:	 0.00035304092751520277


##  Linear Regression

In [49]:
reg = LinearRegression().fit(X, y)
y_pred = reg.predict(X)
mse  = mean_squared_error(y, y_pred)
print("Mean squared error:\t",mse)

Mean squared error:	 0.0017755660228431225


## Multioutput Regression using SGD

In [50]:
model = MultiOutputRegressor(SGDRegressor(loss="squared_error",tol=1e-3)).fit(X, y)
y_pred = model.predict(X)
mse  = mean_squared_error(y, y_pred)
print("Mean squared error:\t",mse)

Mean squared error:	 0.0003713804898648525


### Again but with Max normalization

In [51]:
model = MultiOutputRegressor(SGDRegressor(loss="squared_error",tol=1e-3)).fit(X_scaled, y)
y_pred = model.predict(X_scaled)
mse  = mean_squared_error(y, y_pred)
print("Mean squared error:\t",mse)



Mean squared error:	 0.0217948220161012


## Torch's linear layer

In [52]:
model = t.nn.Linear(40, 4).to(t.float32)
opt = t.optim.SGD(lr=1e-3, params=model.parameters())
loss_fn = t.nn.MSELoss()
for i in range(1001):
     train_loss = loss_fn(y, model(X))
     if i%200==0:
          print(f'Train loss for iteration {i}:\t{train_loss}')
     opt.zero_grad()
     train_loss.backward()
     opt.step()

Train loss for iteration 0:	0.7120311856269836
Train loss for iteration 200:	0.05224146693944931
Train loss for iteration 400:	0.027928924188017845
Train loss for iteration 600:	0.025338636711239815
Train loss for iteration 800:	0.023620178923010826
Train loss for iteration 1000:	0.022054994478821754


### Again but with Max normalization

In [53]:
model = t.nn.Linear(40, 4).to(t.float32)
opt = t.optim.SGD(lr=1e-3, params=model.parameters())
loss_fn = t.nn.MSELoss()
for i in range(1001):
     train_loss = loss_fn(y, model(X_scaled))
     if i%200==0:
          print(f'Train loss for iteration {i}:\t{train_loss}')
     opt.zero_grad()
     train_loss.backward()
     opt.step()

Train loss for iteration 0:	0.5132244229316711
Train loss for iteration 200:	0.4241345524787903
Train loss for iteration 400:	0.3511976897716522
Train loss for iteration 600:	0.2914852201938629
Train loss for iteration 800:	0.24259929358959198
Train loss for iteration 1000:	0.20257702469825745


### Again but using batches 

In [55]:
dataset = TensorDataset(X, y)
train_loader = D.DataLoader(dataset, batch_size=1024, shuffle=False)
model = t.nn.Linear(40, 4).to(t.float32)
opt = t.optim.SGD(lr=1e-3, params=model.parameters())
loss_fn = t.nn.MSELoss()
for j in range(10):
     for i, (b_x, b_y) in enumerate(train_loader):
          train_loss = loss_fn(b_y, model(b_x))
          opt.zero_grad()
          train_loss.backward()
          opt.step()
     print(f'Train loss for epoch {j}:\t{train_loss}')

Train loss for epoch 0:	0.003784812055528164
Train loss for epoch 1:	0.0018951307283714414
Train loss for epoch 2:	0.001179864164441824
Train loss for epoch 3:	0.0008855920750647783
Train loss for epoch 4:	0.0007515850011259317
Train loss for epoch 5:	0.0006834083469584584
Train loss for epoch 6:	0.0006446100305765867
Train loss for epoch 7:	0.0006199977942742407
Train loss for epoch 8:	0.0006027251947671175
Train loss for epoch 9:	0.0005894865025766194
